# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
city_data = pd.read_csv("../WeatherPy/city_data.csv")
city_data.head(8)

,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,carnarvon,-24.87,113.63,19.00,82,90,8.70,AU,1594961110
1,cape town,-33.93,18.42,8.89,75,0,2.10,ZA,1594961158
2,zemio,5.03,25.14,20.29,96,74,0.13,CF,1594961159
3,hilo,19.73,-155.09,27.00,74,20,2.10,US,1594961159
4,nikolskoye,59.70,30.79,15.56,77,86,4.00,RU,1594961159
5,nanortalik,60.14,-45.24,8.77,60,2,2.53,GL,1594961159
6,hithadhoo,-0.60,73.08,29.05,72,95,4.34,MV,1594961160
7,ribeira grande,38.52,-28.70,20.51,80,36,4.78,PT,1594961160


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
